In [1]:
# !ipcluster clean
# !ipcluster stop --all
# !ipcluster list
!lsof -i:24950 | awk 'NR>1{print $2}' | uniq | xargs -I{} kill -9 {}
!nvidia-smi

/bin/bash: nvidia-smi: command not found


In [2]:
# import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
# !echo $CUDA_LAUNCH_BLOCKING

In [3]:
%load_ext autoreload
%autoreload 2
%cd /home/lincong/optimus/test
%load_ext ipytorch
# %ipt cluster -n 4 --engines MPI
# %ipt cluster -n 4 --engines slurm
%ipt client --profile=hgx

/home/lincong/optimus/test


 ccnl-mgmt01.scc.idea: INFO 04:13:44 PM cluster.py:109] {'profile': 'hgx'}


Timestamp('2023-12-19 08:13:45.299640+0000', tz='UTC')
<DirectView [0, 1, 2, 3]>
world_size=4

In [4]:

import os

# os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,4"
from ipytorch import logging

# os.environ["TRANSFORMERS_OFFLINE"] = "1"
# os.environ["HF_DATASETS_OFFLINE"] = "1"
import torch
import torch.distributed as dist
from optimus import mpu
from optimus.utils import init_parallel, release_cuda
import os
from pathlib import Path
import os
import torch
from ipytorch.utils import is_notebook
from tqdm.auto import tqdm

tqdm.pandas()
import sys
from ipytorch.utils import is_notebook
import logging
import deepspeed

ds_logger = logging.getLogger("DeepSpeed")
ds_logger.setLevel(logging.WARNING)  # 只显示 ERROR 级别的日志

%px:   0%|          | 0/4 [00:00<?, ?tasks/s]

[stdout:2] [2023-12-19 16:14:08,765] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


[stdout:1] [2023-12-19 16:14:08,767] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


[stdout:0] [2023-12-19 16:14:08,768] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


[stdout:3] [2023-12-19 16:14:08,770] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [5]:

if not is_notebook():
    from absl import app, flags
    from absl.app import _run_init, parse_flags_with_usage

    FLAGS = flags.FLAGS

    flags.DEFINE_string(
        "experiment_name", None, "name of experiment", short_name="name"
    )
    flags.DEFINE_string("model_path", None, "path to model", short_name="m")
    flags.DEFINE_string("dataset_path", None, "path to dataset", short_name="d")

    args = _run_init(sys.argv, parse_flags_with_usage)

else:

    class FLAGS:
        experiment_name = "RM TRAINING NB"
        model_path = "/data/hf/platypus-13b-mp2"
        # model_path = "/data/hf/platypus-13b-mp2-from-pp"
        dataset_path = "sustech/prm800k-parsed"


PATH = Path("/cognitive_comp/zejianxie/platy-13b-mp2")
# PATH = Path("/data/hf/platypus-13b-mp2")

In [6]:
topo = init_parallel(mp_size=2, pp_size=2)
DS_PATH = Path(FLAGS.dataset_path)
MODEL_PATH = PATH / f"part_{mpu.get_model_parallel_rank()}"

[stderr:2] RANK 2/4 hgx022.scc.idea: WARNING 04:14:17 PM parallel_unit.py:55] No model name is provided, using default name: 0


[stderr:3] RANK 3/4 hgx022.scc.idea: WARNING 04:14:17 PM parallel_unit.py:55] No model name is provided, using default name: 0


[stderr:1] RANK 1/4 hgx022.scc.idea: WARNING 04:14:17 PM parallel_unit.py:55] No model name is provided, using default name: 0


[stderr:0] RANK 0/4 hgx022.scc.idea: WARNING 04:14:17 PM parallel_unit.py:55] No model name is provided, using default name: 0


[stdout:0] > initializing model parallel with size 2
MPU DP: [0]
MPU DP: [1]
MPU DP: [2]
MPU DP: [3]
MPU PP: [0, 2]
MPU PP: [1, 3]
MPU IO: [0, 2]
MPU MP: [0, 1]
MPU MP: [2, 3]


In [7]:

from optimus.hf.llama.llama_packing_pipe import LlamaForCausalLMPipePacking
from optimus.hf.llama.llama import LlamaConfig, LlamaForCausalLM
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "right"
config = LlamaConfig.from_pretrained(MODEL_PATH)
model = (
    LlamaForCausalLMPipePacking.from_mp_pretrained(
        MODEL_PATH,
        torch_dtype=torch.bfloat16,
        config=config,
        topo=topo,
        loss_fn=lambda x: x,
        # save_pp_dir=MODEL_PATH.parent,
    )
    .cuda()
    .eval()
)

[stderr:2] RANK 2/4 hgx022.scc.idea: ERROR 04:14:17 PM attention.py:28] Xformers >= 0.0.21 not found, when use ALIBI, back to fuse softmax
RANK 2/4 hgx022.scc.idea: INFO 04:14:22 PM attention.py:213] Flash attention successfully loaded At layer0!!
Some weights of LlamaForCausalLMPipePacking were not initialized from the model checkpoint at /cognitive_comp/zejianxie/platy-13b-mp2/part_0 and are newly initialized: ['llama.26.transformer_layer.mlp.w2.weight', 'llama.32.transformer_layer.mlp.w3.weight', 'llama.35.transformer_layer.input_layernorm.scale', 'llama.23.transformer_layer.mlp.w3.weight', 'llama.25.transformer_layer.attention.query_key_value.weight', 'llama.25.transformer_layer.mlp.w2.weight', 'llama.31.transformer_layer.post_attention_layernorm.scale', 'llama.35.transformer_layer.mlp.w1.weight', 'llama.26.transformer_layer.input_layernorm.scale', 'llama.39.transformer_layer.post_attention_layernorm.scale', 'llama.21.transformer_layer.attention.dense.weight', 'llama.36.transformer

[stderr:3] RANK 3/4 hgx022.scc.idea: ERROR 04:14:17 PM attention.py:28] Xformers >= 0.0.21 not found, when use ALIBI, back to fuse softmax
RANK 3/4 hgx022.scc.idea: INFO 04:14:22 PM attention.py:213] Flash attention successfully loaded At layer0!!
Some weights of LlamaForCausalLMPipePacking were not initialized from the model checkpoint at /cognitive_comp/zejianxie/platy-13b-mp2/part_1 and are newly initialized: ['llama.34.transformer_layer.mlp.w2.weight', 'llama.22.transformer_layer.mlp.w3.weight', 'llama.40.transformer_layer.input_layernorm.scale', 'llama.28.transformer_layer.input_layernorm.scale', 'llama.34.transformer_layer.mlp.w1.weight', 'llama.21.transformer_layer.mlp.w3.weight', 'llama.40.transformer_layer.mlp.w1.weight', 'llama.22.transformer_layer.input_layernorm.scale', 'llama.32.transformer_layer.mlp.w3.weight', 'llama.28.transformer_layer.post_attention_layernorm.scale', 'llama.30.transformer_layer.attention.query_key_value.weight', 'llama.28.transformer_layer.mlp.w1.weig

[stderr:0] RANK 0/4 hgx022.scc.idea: ERROR 04:14:17 PM attention.py:28] Xformers >= 0.0.21 not found, when use ALIBI, back to fuse softmax
RANK 0/4 hgx022.scc.idea: INFO 04:14:22 PM attention.py:213] Flash attention successfully loaded At layer0!!
RANK 0/4 hgx022.scc.idea: INFO 04:14:22 PM attention.py:213] Flash attention successfully loaded At layer0!!
Some weights of LlamaForCausalLMPipePacking were not initialized from the model checkpoint at /cognitive_comp/zejianxie/platy-13b-mp2/part_0 and are newly initialized: ['llama.12.transformer_layer.attention.dense.weight', 'llama.17.transformer_layer.post_attention_layernorm.scale', 'llama.6.transformer_layer.input_layernorm.scale', 'llama.6.transformer_layer.mlp.w2.weight', 'llama.10.transformer_layer.mlp.w3.weight', 'llama.16.transformer_layer.mlp.w3.weight', 'llama.20.transformer_layer.post_attention_layernorm.scale', 'llama.5.transformer_layer.post_attention_layernorm.scale', 'llama.15.transformer_layer.post_attention_layernorm.scal

[stderr:1] RANK 1/4 hgx022.scc.idea: ERROR 04:14:17 PM attention.py:28] Xformers >= 0.0.21 not found, when use ALIBI, back to fuse softmax
RANK 1/4 hgx022.scc.idea: INFO 04:14:22 PM attention.py:213] Flash attention successfully loaded At layer0!!
RANK 1/4 hgx022.scc.idea: INFO 04:14:22 PM attention.py:213] Flash attention successfully loaded At layer0!!
Some weights of LlamaForCausalLMPipePacking were not initialized from the model checkpoint at /cognitive_comp/zejianxie/platy-13b-mp2/part_1 and are newly initialized: ['llama.11.transformer_layer.mlp.w1.weight', 'llama.11.transformer_layer.mlp.w3.weight', 'llama.15.transformer_layer.attention.dense.weight', 'llama.10.transformer_layer.input_layernorm.scale', 'llama.20.transformer_layer.mlp.w1.weight', 'llama.7.transformer_layer.attention.dense.weight', 'llama.4.transformer_layer.post_attention_layernorm.scale', 'llama.15.transformer_layer.mlp.w1.weight', 'llama.15.transformer_layer.input_layernorm.scale', 'llama.9.transformer_layer.at

[stdout:0] SEED_LAYERS=False BASE_SEED=1234 SEED_FN=None
Using topology: {ProcessCoord(pipe=0, data=0, model=0): 0, ProcessCoord(pipe=0, data=0, model=1): 1, ProcessCoord(pipe=1, data=0, model=0): 2, ProcessCoord(pipe=1, data=0, model=1): 3}
stage=0 layers=21
     0: LlamaInputLayerPacking
     1: LlamaLayerPacking
     2: LlamaLayerPacking
     3: LlamaLayerPacking
     4: LlamaLayerPacking
     5: LlamaLayerPacking
     6: LlamaLayerPacking
     7: LlamaLayerPacking
     8: LlamaLayerPacking
     9: LlamaLayerPacking
    10: LlamaLayerPacking
    11: LlamaLayerPacking
    12: LlamaLayerPacking
    13: LlamaLayerPacking
    14: LlamaLayerPacking
    15: LlamaLayerPacking
    16: LlamaLayerPacking
    17: LlamaLayerPacking
    18: LlamaLayerPacking
    19: LlamaLayerPacking
    20: LlamaLayerPacking
stage=1 layers=22
    21: LlamaLayerPacking
    22: LlamaLayerPacking
    23: LlamaLayerPacking
    24: LlamaLayerPacking
    25: LlamaLayerPacking
    26: LlamaLayerPacking
    27: LlamaLa

...

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

[output:0]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

[output:2]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [8]:

from optimus.utils import release_cuda

release_cuda()

In [9]:

question = "Introduce prime number"
header = f"""### Instruction:\n{question}"""
mid = "\n### Response:"
prompt = f"{header}{mid}"
print(prompt)

[stdout:0] ### Instruction:
Introduce prime number
### Response:


[stdout:1] ### Instruction:
Introduce prime number
### Response:


[stdout:2] ### Instruction:
Introduce prime number
### Response:


[stdout:3] ### Instruction:
Introduce prime number
### Response:


In [10]:

question = "Hello, who are you?"
header = f"""### Instruction:\n{question}"""
mid = "\n### Response:"
prompt1 = f"{header}{mid}"
print(prompt1)

[stdout:1] ### Instruction:
Hello, who are you?
### Response:


[stdout:0] ### Instruction:
Hello, who are you?
### Response:


[stdout:2] ### Instruction:
Hello, who are you?
### Response:


[stdout:3] ### Instruction:
Hello, who are you?
### Response:


In [11]:

i1 = tokenizer.encode(prompt, return_tensors="pt")[0].cuda()
i2 = tokenizer.encode(prompt1, return_tensors="pt")[0].cuda()
cu_len = [0, len(i1), len(i2)]
cu_len = torch.tensor(cu_len, dtype=torch.int32, device="cuda")
cu_len = torch.cumsum(cu_len, dim=0)
inputs = torch.cat([i1, i2], dim=0)
inputs

Out[0:8]: 
tensor([    1,   835,  2799,  4080, 29901,    13,  2928,  3518,   346,  6019,
         1353,    13,  2277, 29937, 13291, 29901,     1,   835,  2799,  4080,
        29901,    13, 10994, 29892,  1058,   526,   366, 29973,    13,  2277,
        29937, 13291, 29901], device='cuda:0')

Out[2:8]: 
tensor([    1,   835,  2799,  4080, 29901,    13,  2928,  3518,   346,  6019,
         1353,    13,  2277, 29937, 13291, 29901,     1,   835,  2799,  4080,
        29901,    13, 10994, 29892,  1058,   526,   366, 29973,    13,  2277,
        29937, 13291, 29901], device='cuda:2')

Out[1:8]: 
tensor([    1,   835,  2799,  4080, 29901,    13,  2928,  3518,   346,  6019,
         1353,    13,  2277, 29937, 13291, 29901,     1,   835,  2799,  4080,
        29901,    13, 10994, 29892,  1058,   526,   366, 29973,    13,  2277,
        29937, 13291, 29901], device='cuda:1')

Out[3:8]: 
tensor([    1,   835,  2799,  4080, 29901,    13,  2928,  3518,   346,  6019,
         1353,    13,  2277, 29937, 13291, 29901,     1,   835,  2799,  4080,
        29901,    13, 10994, 29892,  1058,   526,   366, 29973,    13,  2277,
        29937, 13291, 29901], device='cuda:3')

In [12]:

with torch.no_grad():
    args = (
        inputs.cuda(),
        cu_len.to(dtype=torch.int32, device="cuda"),
        torch.tensor(3, dtype=torch.int64, device="cuda"),
    )
    out = model.pipe_forward(args)

out_logits = out[[int(cu_len[1] - 1), int(cu_len[2] - 1)]]
out_tokens = torch.argmax(out_logits, dim=-1)

[stderr:3] [W ProcessGroupNCCL.cpp:1856] Warning: 0NCCL_AVOID_RECORD_STREAMS=1 has no effect for point-to-point collectives. (function operator())


[stderr:2] [W ProcessGroupNCCL.cpp:1856] Warning: 0NCCL_AVOID_RECORD_STREAMS=1 has no effect for point-to-point collectives. (function operator())


%px:   0%|          | 0/4 [00:00<?, ?tasks/s]

[stderr:1] [W ProcessGroupNCCL.cpp:1856] Warning: 0NCCL_AVOID_RECORD_STREAMS=1 has no effect for point-to-point collectives. (function operator())


[stderr:0] [W ProcessGroupNCCL.cpp:1856] Warning: 0NCCL_AVOID_RECORD_STREAMS=1 has no effect for point-to-point collectives. (function operator())


In [13]:

i1 = torch.cat([i1, out_tokens[0].view(-1)], dim=0)
i2 = torch.cat([i2, out_tokens[1].view(-1)], dim=0)
print("==============================================================")
print(tokenizer.decode(i1.tolist()))
print("==============================================================")
print(tokenizer.decode(i2.tolist()))
print("==============================================================")
inputs = torch.cat([i1, i2], dim=0)

cu_len = [0, len(i1), len(i2)]
cu_len = torch.tensor(cu_len, dtype=torch.int32, device="cuda")
cu_len = torch.cumsum(cu_len, dim=0)
with torch.no_grad():
    args = (
        inputs.cuda(),
        cu_len.to(dtype=torch.int32, device="cuda"),
        torch.tensor(3, dtype=torch.int64, device="cuda"),
    )
    out = model.pipe_forward(args)
out_logits = out[[int(cu_len[1] - 1), int(cu_len[2] - 1)]]
out_tokens = torch.argmax(out_logits, dim=-1)

[stdout:2] ==============================================================
<s> ### Instruction:
Introduce prime number
### Response: A
<s> ### Instruction:
Hello, who are you?
### Response: I


[stdout:0] ==============================================================
<s> ### Instruction:
Introduce prime number
### Response: A
<s> ### Instruction:
Hello, who are you?
### Response: I


[stdout:3] ==============================================================
<s> ### Instruction:
Introduce prime number
### Response: A
<s> ### Instruction:
Hello, who are you?
### Response: I


[stdout:1] ==============================================================
<s> ### Instruction:
Introduce prime number
### Response: A
<s> ### Instruction:
Hello, who are you?
### Response: I
